In [ ]:
import os
from PIL import Image
import numpy as np
import cv2
import torch
from torchvision import transforms
from torchvision.models.segmentation import deeplabv3_resnet101
from torchvision.models.segmentation.deeplabv3 import DeepLabHead
from torch.cuda.amp import autocast
import matplotlib.pyplot as plt

In [ ]:
# Settings
DEVICE        = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
WEIGHTS_PATH  = '/content/drive/MyDrive/pt_data/model_weights/deeplabv3_ep30.pth'
NUM_CLASSES   = 2
TOP_WIDTH_FRAC= 0.3
BOTTOM_FRAC   = 0.5

In [ ]:
# Transform for input
seg_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

In [ ]:
# Model definition and weight loading
def get_deeplab_model(num_classes):
    model = deeplabv3_resnet101(pretrained=False)
    model.classifier = DeepLabHead(2048, num_classes)
    return model

model = get_deeplab_model(NUM_CLASSES).to(DEVICE)
model.load_state_dict(torch.load(WEIGHTS_PATH, map_location=DEVICE))
model.eval()

In [ ]:
# ROI warp utilities
def get_road_mask(img, margin=0.1):
    h,w = img.shape[:2]
    x0,x1 = int(w*margin), int(w*(1-margin))
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    m = cv2.inRange(hsv, (0,0,60), (180,50,200))
    m = cv2.morphologyEx(m, cv2.MORPH_CLOSE,
                        cv2.getStructuringElement(cv2.MORPH_RECT,(15,15)))
    m[:,:x0]=0; m[:,x1:]=0
    return m

In [ ]:
def compute_bottom_y(mask, bottom_frac=0.5):
    h,_ = mask.shape
    start = int(h*(1-bottom_frac))
    ys = np.where(mask[start:,:].any(1))[0]
    return (start+ys.max()) if ys.size else h

In [ ]:
def compute_bottom_bounds(mask, yb, sample_n=5):
    xs=[]
    for dy in range(sample_n):
        y = max(0,int(yb)-1-dy)
        nz = np.where(mask[y]>0)[0]
        if nz.size: xs.append([nz.min(), nz.max()])
    if not xs: return 0.0, float(mask.shape[1])
    arr = np.array(xs,np.float32)
    return float(arr[:,0].mean()), float(arr[:,1].mean())

In [ ]:
def estimate_vp_all(img):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray,50,150)
    lines = cv2.HoughLinesP(edges,1,np.pi/180,100,
                             minLineLength=50,maxLineGap=10)
    if lines is None: return img.shape[1]/2, img.shape[0]*0.4
    from itertools import combinations
    pts=[]
    for (x1,y1,x2,y2),(x3,y3,x4,y4) in combinations([l[0] for l in lines],2):
        D=(x1-x2)*(y3-y4)-(y1-y2)*(x3-x4)
        if abs(D)<1e-6: continue
        px = ((x1*y2-y1*x2)*(x3-x4)-(x1-x2)*(x3*y4-y3*x4))/D
        py = ((x1*y2-y1*x2)*(y3-y4)-(y1-y2)*(x3*y4-y3*x4))/D
        if -img.shape[1]<px<2*img.shape[1] and -img.shape[0]<py<2*img.shape[0]:
            pts.append((px,py))
    if not pts: return img.shape[1]/2, img.shape[0]*0.4
    return tuple(np.mean(pts, axis=0))

In [ ]:
 def warp_roi(img, mask_bin, top_width_frac=TOP_WIDTH_FRAC, bottom_frac=BOTTOM_FRAC):
    """
    Args:
        img (np.ndarray): Original BGR image of shape (H, W, 3).
        mask_bin (np.ndarray): Binary mask of shape (H, W), dtype uint8 with values 0 or 255.
        top_width_frac (float): Fractional width at image top for ROI.
        bottom_frac (float): Fractional bottom region height for ROI baseline.
    Returns:
        bird_img (np.ndarray): Warped and squared BGR image.
        bird_mask (np.ndarray): Warped and squared binary mask.
    """
    h,w = img.shape[:2]
    road = get_road_mask(img)
    yb = compute_bottom_y(road, bottom_frac)
    xl,xr = compute_bottom_bounds(road, yb)
    vp_x,vp_y = estimate_vp_all(img)
    bw = xr - xl; tw = bw * top_width_frac; mid = (xl + xr)/2.0
    src = np.array([[xl,yb],[xr,yb],[mid+tw/2,vp_y],[mid-tw/2,vp_y]],dtype=np.float32)
    ys,xs = np.where(mask_bin>0)
    if xs.size:
        src[0,0] = min(src[0,0], xs.min())
        src[1,0] = max(src[1,0], xs.max())
    bird_w = max(int(src[1,0]-src[0,0]),1)
    bird_h = max(int((src[0,1]-src[2,1])/top_width_frac),1)
    dst = np.array([[0,bird_h],[bird_w,bird_h],[bird_w,0],[0,0]],dtype=np.float32)
    M = cv2.getPerspectiveTransform(src,dst)
    bird_img  = cv2.warpPerspective(img,M,(bird_w,bird_h))
    bird_mask = cv2.warpPerspective(mask_bin,M,(bird_w,bird_h))
    # square padding
    h_b,w_b = bird_img.shape[:2]
    if w_b< h_b:
        pad = h_b-w_b; left=pad//2; right=pad-left
        bird_img  = cv2.copyMakeBorder(bird_img,0,0,left,right,cv2.BORDER_CONSTANT,value=[0,0,0])
        bird_mask = cv2.copyMakeBorder(bird_mask,0,0,left,right,cv2.BORDER_CONSTANT,value=0)
    elif w_b> h_b:
        pad = w_b-h_b; top=pad//2; bot=pad-top
        bird_img  = cv2.copyMakeBorder(bird_img,top,bot,0,0,cv2.BORDER_CONSTANT,value=[0,0,0])
        bird_mask = cv2.copyMakeBorder(bird_mask,top,bot,0,0,cv2.BORDER_CONSTANT,value=0)
    return bird_img, bird_mask

In [ ]:
# Inference & ROI Warp
def infer_and_warp(img_np):
    img_pil = Image.fromarray(img_np[...,::-1])
    x = seg_tf(img_pil).unsqueeze(0).to(DEVICE)
    with torch.no_grad(), autocast():
        out = model(x)['out']
    pred = out.argmax(1).squeeze(0).cpu().numpy().astype(np.uint8)
    mask_bin = (pred>0).astype(np.uint8)*255
    return warp_roi(img_np, mask_bin)

In [ ]:
# Example Usage:
# if __name__ == '__main__':
#     img = cv2.imread('path/to/image.jpg')
#     warped_img, warped_mask = infer_and_warp(img)
#     plt.subplot(1,2,1); plt.imshow(cv2.cvtColor(warped_img,cv2.COLOR_BGR2RGB)); plt.axis('off')
#     plt.subplot(1,2,2); plt.imshow(warped_mask, cmap='gray'); plt.axis('off')
#     plt.show()
